# Experiment: MSFT 1-Minute GRU Forecast (Anti-Linear v3)

This notebook implements the six requested anti-linear fixes plus volume/order-flow integration:
1. Market-calendar sessionization (no hard 390 assumption) + high-fill session drop
2. Imputation policy update: drop windows only if target horizon touches imputed bars; add `imputedFracWindow` feature
3. Lookback sweep (`96/160/256`) before final fold runs
4. Probabilistic decoder (`mu` + `sigma`) with Gaussian NLL
5. Relaxed clipping + variance-matching loss
6. Aggressive teacher-forcing decay to near-zero by epoch 10


In [ ]:
# Optional dependency bootstrap
import importlib.util
import subprocess
import sys

required = {
    'alpaca': 'alpaca-py',
    'numpy': 'numpy',
    'pandas': 'pandas',
    'matplotlib': 'matplotlib',
    'pandas_market_calendars': 'pandas-market-calendars',
}

missing = [pkg for mod, pkg in required.items() if importlib.util.find_spec(mod) is None]
if missing:
    print('Installing missing packages:', missing)
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', *missing])
else:
    print('All required third-party packages are already installed.')


In [ ]:
# Setup
from __future__ import annotations

import copy
import os
import random
import time
from datetime import datetime, timedelta, timezone

import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import torch
import torch.nn as nn
from alpaca.data.enums import DataFeed
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from IPython.display import display
from matplotlib import pyplot as plt
from matplotlib.patches import Patch, Rectangle
from torch.utils.data import DataLoader, Dataset

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.benchmark = True
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Using device: {DEVICE}')
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
    print('CUDA:', torch.version.cuda)


## Configuration

Defaults are tuned for free-plan IEX while reducing linear-collapse behavior.


In [ ]:
# Configuration
SYMBOL = 'MSFT'
LOOKBACK_DAYS = 60

OHLC_COLS = ['Open', 'High', 'Low', 'Close']
RAW_COLS = OHLC_COLS + ['Volume', 'TradeCount', 'VWAP']

BASE_FEATURE_COLS = [
    'rOpen',
    'rHigh',
    'rLow',
    'rClose',
    'logVolChange',
    'logTradeCountChange',
    'vwapDelta',
    'rangeFrac',
    'orderFlowProxy',
    'tickPressure',
]
INPUT_EXTRA_COL = 'imputedFracWindow'

HORIZON = 15
TRAIN_RATIO = 0.70
VAL_RATIO = 0.15

LOOKBACK_CANDIDATES = [96, 160, 256]
DEFAULT_LOOKBACK = 160
ENABLE_LOOKBACK_SWEEP = True

# Model capacity / regularization
HIDDEN_SIZE = 256
NUM_LAYERS = 3
DROPOUT = 0.0
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
BATCH_SIZE = 256

# Training schedules
SWEEP_MAX_EPOCHS = 12
SWEEP_PATIENCE = 4
FINAL_MAX_EPOCHS = 35
FINAL_PATIENCE = 8
TF_START = 1.0
TF_END = 0.05
TF_RAMP_EPOCHS = 10

# Probabilistic + auxiliary objectives
HUBER_DELTA = 1.0
NLL_WEIGHTS = np.array([1.0, 1.0, 1.0, 2.0, 0.4, 0.4, 0.3, 0.5, 0.6, 0.6], dtype=np.float32)
DIR_LOSS_WEIGHT = 0.20
VAR_MATCH_WEIGHT = 0.15

# Clipping / sampling
APPLY_CLIPPING = True
CLIP_NON_OHLC = False
CLIP_QUANTILES = (0.0005, 0.9995)
INFER_SAMPLE_TEMPERATURE = 1.0

# Diagnostics
DIRECTION_EPS = 0.02
STD_RATIO_TARGET_MIN = 0.60

# Session and feed
ALPACA_FEED = os.getenv('ALPACA_FEED', 'iex').strip().lower()
SESSION_TZ = 'America/New_York'
REQUEST_CHUNK_DAYS = 5
MAX_REQUESTS_PER_MINUTE = 120
MAX_RETRIES = 5
MAX_SESSION_FILL_RATIO = 0.08

print({
    'symbol': SYMBOL,
    'lookback_days': LOOKBACK_DAYS,
    'lookback_candidates': LOOKBACK_CANDIDATES,
    'horizon': HORIZON,
    'feed': ALPACA_FEED,
    'max_session_fill_ratio': MAX_SESSION_FILL_RATIO,
    'clip_quantiles': CLIP_QUANTILES,
    'tf_ramp_epochs': TF_RAMP_EPOCHS,
    'device': str(DEVICE),
})


In [ ]:
# Data pull + calendar-aware sessionization
class RequestPacer:
    def __init__(self, max_calls_per_minute: int):
        if max_calls_per_minute <= 0:
            raise ValueError('max_calls_per_minute must be > 0')
        self.min_interval = 60.0 / float(max_calls_per_minute)
        self.last_call_ts = 0.0

    def wait(self) -> None:
        now = time.monotonic()
        elapsed = now - self.last_call_ts
        if elapsed < self.min_interval:
            time.sleep(self.min_interval - elapsed)
        self.last_call_ts = time.monotonic()


def _require_alpaca_credentials() -> tuple[str, str]:
    api_key = os.getenv('ALPACA_API_KEY')
    secret_key = os.getenv('ALPACA_SECRET_KEY')
    if not api_key or not secret_key:
        raise RuntimeError('Missing ALPACA_API_KEY / ALPACA_SECRET_KEY.')
    return api_key, secret_key


def _resolve_feed(feed_name: str) -> DataFeed:
    mapping = {'iex': DataFeed.IEX, 'sip': DataFeed.SIP, 'delayed_sip': DataFeed.DELAYED_SIP}
    k = feed_name.strip().lower()
    if k not in mapping:
        raise ValueError(f'Unsupported ALPACA_FEED={feed_name!r}. Use one of: {list(mapping)}')
    return mapping[k]


def fetch_bars_alpaca(symbol: str, lookback_days: int) -> tuple[pd.DataFrame, int]:
    api_key, secret_key = _require_alpaca_credentials()
    client = StockHistoricalDataClient(api_key=api_key, secret_key=secret_key)

    feed = _resolve_feed(ALPACA_FEED)
    pacer = RequestPacer(MAX_REQUESTS_PER_MINUTE)

    end_ts = datetime.now(timezone.utc).replace(second=0, microsecond=0)
    if ALPACA_FEED in {'sip', 'delayed_sip'}:
        end_ts = end_ts - timedelta(minutes=20)
    start_ts = end_ts - timedelta(days=lookback_days)

    parts = []
    cursor = start_ts
    calls = 0

    while cursor < end_ts:
        chunk_end = min(cursor + timedelta(days=REQUEST_CHUNK_DAYS), end_ts)
        chunk = None

        for attempt in range(1, MAX_RETRIES + 1):
            pacer.wait()
            calls += 1
            try:
                req = StockBarsRequest(
                    symbol_or_symbols=[symbol],
                    timeframe=TimeFrame.Minute,
                    start=cursor,
                    end=chunk_end,
                    feed=feed,
                    limit=10000,
                )
                chunk = client.get_stock_bars(req).df
                break
            except Exception as exc:
                msg = str(exc).lower()
                if ('429' in msg or 'rate limit' in msg) and attempt < MAX_RETRIES:
                    backoff = min(2 ** attempt, 30)
                    print(f'Rate-limited; sleeping {backoff}s (attempt {attempt}/{MAX_RETRIES}).')
                    time.sleep(backoff)
                    continue
                if ('subscription' in msg or 'forbidden' in msg) and ALPACA_FEED != 'iex':
                    raise RuntimeError('Feed unavailable for account. Use ALPACA_FEED=iex or upgrade subscription.') from exc
                raise

        if chunk is not None and not chunk.empty:
            d = chunk.reset_index().rename(
                columns={
                    'timestamp': 'Datetime',
                    'open': 'Open',
                    'high': 'High',
                    'low': 'Low',
                    'close': 'Close',
                    'volume': 'Volume',
                    'trade_count': 'TradeCount',
                    'vwap': 'VWAP',
                }
            )
            if 'Volume' not in d.columns:
                d['Volume'] = 0.0
            if 'TradeCount' not in d.columns:
                d['TradeCount'] = 0.0
            if 'VWAP' not in d.columns:
                d['VWAP'] = d['Close']

            need = ['Datetime'] + RAW_COLS
            missing = [c for c in need if c not in d.columns]
            if missing:
                raise RuntimeError(f'Alpaca response missing columns: {missing}')

            d['Datetime'] = pd.to_datetime(d['Datetime'], utc=True)
            d = d[need].dropna(subset=OHLC_COLS).set_index('Datetime').sort_index()
            parts.append(d)

        cursor = chunk_end

    if not parts:
        raise RuntimeError('No bars returned from Alpaca.')

    out = pd.concat(parts, axis=0).sort_index()
    out = out[~out.index.duplicated(keep='last')]
    return out.astype(np.float32), calls


def sessionize_with_calendar(df_utc: pd.DataFrame) -> tuple[pd.DataFrame, dict]:
    if df_utc.empty:
        raise RuntimeError('Input bars are empty.')

    idx = pd.DatetimeIndex(df_utc.index)
    if idx.tz is None:
        idx = idx.tz_localize('UTC')
    else:
        idx = idx.tz_convert('UTC')
    df_utc = df_utc.copy()
    df_utc.index = idx

    cal = mcal.get_calendar('XNYS')
    sched = cal.schedule(
        start_date=(idx.min() - pd.Timedelta(days=2)).date(),
        end_date=(idx.max() + pd.Timedelta(days=2)).date(),
    )

    pieces = []
    kept = 0
    dropped_high_fill = 0
    dropped_empty = 0
    fill_ratios = []
    expected_lengths = []

    for _, row in sched.iterrows():
        open_ts = pd.Timestamp(row['market_open'])
        close_ts = pd.Timestamp(row['market_close'])
        if open_ts.tzinfo is None:
            open_ts = open_ts.tz_localize('UTC')
        else:
            open_ts = open_ts.tz_convert('UTC')
        if close_ts.tzinfo is None:
            close_ts = close_ts.tz_localize('UTC')
        else:
            close_ts = close_ts.tz_convert('UTC')

        exp_idx = pd.date_range(open_ts, close_ts, freq='1min', inclusive='left')
        if len(exp_idx) == 0:
            continue

        day = df_utc[(df_utc.index >= open_ts) & (df_utc.index < close_ts)][RAW_COLS].copy()
        day = day.reindex(exp_idx)

        imputed = day[OHLC_COLS].isna().any(axis=1).to_numpy()
        fill_ratio = float(imputed.mean())

        if fill_ratio >= 1.0:
            dropped_empty += 1
            continue

        if fill_ratio > MAX_SESSION_FILL_RATIO:
            dropped_high_fill += 1
            continue

        day[OHLC_COLS + ['VWAP']] = day[OHLC_COLS + ['VWAP']].ffill().bfill()
        if day['VWAP'].isna().all():
            day['VWAP'] = day['Close']
        else:
            day['VWAP'] = day['VWAP'].fillna(day['Close'])

        day['Volume'] = day['Volume'].fillna(0.0)
        day['TradeCount'] = day['TradeCount'].fillna(0.0)
        day['is_imputed'] = imputed.astype(np.int8)

        if day[RAW_COLS].isna().any().any():
            raise RuntimeError('NaNs remain after per-session fill.')

        pieces.append(day)
        kept += 1
        fill_ratios.append(fill_ratio)
        expected_lengths.append(len(exp_idx))

    if not pieces:
        raise RuntimeError('No sessions kept after calendar filtering and fill-ratio threshold.')

    out = pd.concat(pieces, axis=0).sort_index()

    out.index = out.index.tz_convert(SESSION_TZ).tz_localize(None)

    meta = {
        'calendar_sessions_total': int(len(sched)),
        'kept_sessions': int(kept),
        'dropped_empty_sessions': int(dropped_empty),
        'dropped_high_fill_sessions': int(dropped_high_fill),
        'avg_fill_ratio_kept': float(np.mean(fill_ratios)) if fill_ratios else float('nan'),
        'max_fill_ratio_kept': float(np.max(fill_ratios)) if fill_ratios else float('nan'),
        'avg_session_minutes_kept': float(np.mean(expected_lengths)) if expected_lengths else float('nan'),
    }
    return out.astype(np.float32), meta


raw_df_utc, api_calls = fetch_bars_alpaca(SYMBOL, LOOKBACK_DAYS)
price_df, session_meta = sessionize_with_calendar(raw_df_utc)

print(f'Raw rows from Alpaca: {len(raw_df_utc):,}')
print(f'Sessionized rows kept: {len(price_df):,}')
print(f'Alpaca API calls: {api_calls} (<= {MAX_REQUESTS_PER_MINUTE}/min target)')
print('Session meta:', session_meta)

min_needed = max(LOOKBACK_CANDIDATES) + HORIZON + 1000
if len(price_df) < min_needed:
    raise RuntimeError(f'Not enough rows after session filtering ({len(price_df)}). Need at least {min_needed}.')

display(price_df.head(3))
display(price_df.tail(3))


In [ ]:
# Feature engineering (volume/order-flow included)
def enforce_candle_validity(ohlc: np.ndarray) -> np.ndarray:
    out = np.asarray(ohlc, dtype=np.float32).copy()
    o = out[:, 0]
    h = out[:, 1]
    l = out[:, 2]
    c = out[:, 3]
    out[:, 1] = np.maximum.reduce([h, o, c])
    out[:, 2] = np.minimum.reduce([l, o, c])
    return out


def returns_to_prices_seq(return_ohlc: np.ndarray, last_close: float) -> np.ndarray:
    seq = []
    prev_close = float(last_close)
    for rO, rH, rL, rC in np.asarray(return_ohlc, dtype=np.float32):
        o = prev_close * np.exp(float(rO))
        h = prev_close * np.exp(float(rH))
        l = prev_close * np.exp(float(rL))
        c = prev_close * np.exp(float(rC))
        cand = enforce_candle_validity(np.array([[o, h, l, c]], dtype=np.float32))[0]
        seq.append(cand)
        prev_close = float(cand[3])
    return np.asarray(seq, dtype=np.float32)


def one_step_returns_to_prices_batch(return_ohlc: np.ndarray, prev_close: np.ndarray) -> np.ndarray:
    r = np.asarray(return_ohlc, dtype=np.float32)
    p = np.asarray(prev_close, dtype=np.float32)
    out = np.stack([
        p * np.exp(r[:, 0]),
        p * np.exp(r[:, 1]),
        p * np.exp(r[:, 2]),
        p * np.exp(r[:, 3]),
    ], axis=1).astype(np.float32)
    return enforce_candle_validity(out)


def build_feature_frame(df: pd.DataFrame) -> pd.DataFrame:
    eps = 1e-9

    prev_close = df['Close'].shift(1)
    prev_vol = df['Volume'].shift(1)
    prev_tc = df['TradeCount'].shift(1)

    row_imputed = (df['is_imputed'].astype(bool) | df['is_imputed'].shift(1).fillna(0).astype(bool))

    valid = prev_close.notna() & prev_vol.notna() & prev_tc.notna()
    c = df.loc[valid]
    base = prev_close.loc[valid]

    out = pd.DataFrame(index=c.index, dtype=np.float32)

    out['rOpen'] = np.log(c['Open'] / (base + eps))
    out['rHigh'] = np.log(c['High'] / (base + eps))
    out['rLow'] = np.log(c['Low'] / (base + eps))
    out['rClose'] = np.log(c['Close'] / (base + eps))

    out['logVolChange'] = np.log((c['Volume'] + 1.0) / (prev_vol.loc[valid] + 1.0))
    out['logTradeCountChange'] = np.log((c['TradeCount'] + 1.0) / (prev_tc.loc[valid] + 1.0))
    out['vwapDelta'] = np.log((c['VWAP'] + eps) / (c['Close'] + eps))
    out['rangeFrac'] = (c['High'] - c['Low']) / (base + eps)

    signed_body = (c['Close'] - c['Open']) / ((c['High'] - c['Low']) + eps)
    out['orderFlowProxy'] = signed_body * np.log1p(c['Volume'])
    out['tickPressure'] = np.sign(c['Close'] - c['Open']) * np.log1p(c['TradeCount'])

    out['row_imputed'] = row_imputed.loc[valid].astype(np.int8).to_numpy()
    out['prev_close'] = base.astype(np.float32).to_numpy()

    return out.astype(np.float32)


feat_df = build_feature_frame(price_df)
print('Feature rows:', len(feat_df))
print('Rows marked imputed-sensitive:', int(feat_df['row_imputed'].sum()))

# Transform round-trip check
rt_n = min(2000, len(price_df))
sub = price_df.iloc[:rt_n].copy()
sub_feat = build_feature_frame(sub)
ret = sub_feat[['rOpen', 'rHigh', 'rLow', 'rClose']].to_numpy(np.float32)
recon = returns_to_prices_seq(ret, float(sub['Close'].iloc[0]))
actual = sub[OHLC_COLS].iloc[1:].to_numpy(np.float32)

rt_max = float(np.max(np.abs(recon - actual)))
rt_mean = float(np.mean(np.abs(recon - actual)))
print({'roundtrip_max_abs_error': rt_max, 'roundtrip_mean_abs_error': rt_mean})
assert rt_max < 2e-3, 'Round-trip transform error too large.'

display(feat_df[BASE_FEATURE_COLS + ['row_imputed', 'prev_close']].head(3))


In [ ]:
# Windowing/splits + lookback sweep helpers
def fit_standardizer(train_values: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    mean = train_values.mean(axis=0)
    std = train_values.std(axis=0)
    std = np.where(std < 1e-8, 1.0, std)
    return mean.astype(np.float32), std.astype(np.float32)


def apply_standardizer(values: np.ndarray, mean: np.ndarray, std: np.ndarray) -> np.ndarray:
    return ((values - mean) / std).astype(np.float32)


def undo_standardizer(values: np.ndarray, mean: np.ndarray, std: np.ndarray) -> np.ndarray:
    return (values * std + mean).astype(np.float32)


def split_points(n_rows: int) -> tuple[int, int]:
    tr = int(n_rows * TRAIN_RATIO)
    va = int(n_rows * (TRAIN_RATIO + VAL_RATIO))
    return tr, va


def build_walkforward_slices(price_df_full: pd.DataFrame) -> list[tuple[str, int, int]]:
    n = len(price_df_full)
    span = int(round(n * 0.85))
    shift = max(1, n - span)

    cands = [('slice_1', 0, min(span, n)), ('slice_2', shift, min(shift + span, n))]

    out = []
    seen = set()
    for name, a, b in cands:
        key = (a, b)
        if key in seen:
            continue
        if b - a < max(LOOKBACK_CANDIDATES) + HORIZON + 1200:
            continue
        out.append((name, a, b))
        seen.add(key)

    if not out:
        raise RuntimeError('Unable to create walk-forward slices.')
    return out


def make_multistep_windows(
    base_scaled: np.ndarray,
    base_raw: np.ndarray,
    row_imputed: np.ndarray,
    starts_prev_close: np.ndarray,
    window: int,
    horizon: int,
) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, int]:
    X = []
    y_s = []
    y_r = []
    starts = []
    prev_close = []
    dropped_target_imputed = 0

    n = len(base_scaled)
    for i in range(window, n - horizon + 1):
        # Item 2: drop only if TARGET horizon touches imputed rows.
        if row_imputed[i : i + horizon].any():
            dropped_target_imputed += 1
            continue

        xb = base_scaled[i - window : i]

        # Item 2: add input feature `imputedFracWindow` (fraction in input window).
        imp_frac = float(row_imputed[i - window : i].mean())
        imp_col = np.full((window, 1), imp_frac, dtype=np.float32)
        xb_aug = np.concatenate([xb, imp_col], axis=1).astype(np.float32)

        X.append(xb_aug)
        y_s.append(base_scaled[i : i + horizon])
        y_r.append(base_raw[i : i + horizon])
        starts.append(i)
        prev_close.append(starts_prev_close[i])

    return (
        np.asarray(X, dtype=np.float32),
        np.asarray(y_s, dtype=np.float32),
        np.asarray(y_r, dtype=np.float32),
        np.asarray(starts, dtype=np.int64),
        np.asarray(prev_close, dtype=np.float32),
        dropped_target_imputed,
    )


class MultiStepDataset(Dataset):
    def __init__(self, X: np.ndarray, y_s: np.ndarray, y_r: np.ndarray):
        self.X = torch.from_numpy(X).float()
        self.y_s = torch.from_numpy(y_s).float()
        self.y_r = torch.from_numpy(y_r).float()

    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, idx: int):
        return self.X[idx], self.y_s[idx], self.y_r[idx]


slices = build_walkforward_slices(price_df)
print('Walk-forward slices:', slices)


In [ ]:
# Model + losses (probabilistic seq2seq)
class Seq2SeqProbGRU(nn.Module):
    def __init__(self, input_size: int, output_size: int, hidden_size: int, num_layers: int, dropout: float, horizon: int):
        super().__init__()
        self.horizon = horizon
        self.output_size = output_size

        self.encoder = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )

        self.decoder_cell = nn.GRUCell(output_size, hidden_size)
        self.mu_head = nn.Linear(hidden_size, output_size)
        self.log_sigma_head = nn.Linear(hidden_size, output_size)
        self.direction_head = nn.Linear(hidden_size, 1)

    def forward(
        self,
        x: torch.Tensor,
        y_teacher: torch.Tensor | None = None,
        teacher_forcing_ratio: float = 0.0,
    ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        _, h = self.encoder(x)
        h_dec = h[-1]

        dec_input = x[:, -1, : self.output_size]

        mu_seq = []
        log_sigma_seq = []
        dir_seq = []

        for t in range(self.horizon):
            h_dec = self.decoder_cell(dec_input, h_dec)

            mu = self.mu_head(h_dec)
            log_sigma = torch.clamp(self.log_sigma_head(h_dec), min=-5.0, max=2.0)
            dir_logit = self.direction_head(h_dec).squeeze(-1)

            mu_seq.append(mu.unsqueeze(1))
            log_sigma_seq.append(log_sigma.unsqueeze(1))
            dir_seq.append(dir_logit.unsqueeze(1))

            if y_teacher is not None:
                sample_like = mu + 0.30 * torch.exp(log_sigma) * torch.randn_like(mu)

                if teacher_forcing_ratio >= 1.0:
                    dec_input = y_teacher[:, t, :]
                elif teacher_forcing_ratio <= 0.0:
                    dec_input = sample_like
                else:
                    m = (torch.rand(x.size(0), device=x.device) < teacher_forcing_ratio).unsqueeze(1)
                    dec_input = torch.where(m, y_teacher[:, t, :], sample_like)
            else:
                dec_input = mu

        return (
            torch.cat(mu_seq, dim=1),
            torch.cat(log_sigma_seq, dim=1),
            torch.cat(dir_seq, dim=1),
        )


class WeightedGaussianNLL(nn.Module):
    def __init__(self, weights: np.ndarray):
        super().__init__()
        w = torch.as_tensor(weights, dtype=torch.float32).view(1, 1, -1)
        self.register_buffer('weights', w)

    def forward(self, mu: torch.Tensor, log_sigma: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        sigma = torch.exp(log_sigma)
        z = (target - mu) / sigma
        nll = 0.5 * (z * z + 2.0 * log_sigma)
        return (nll * self.weights).mean()


def tf_ratio_for_epoch(epoch: int) -> float:
    # Item 6: aggressive decay to near-zero by epoch 10.
    if epoch <= TF_RAMP_EPOCHS:
        if TF_RAMP_EPOCHS <= 1:
            return float(TF_END)
        p = (epoch - 1) / float(TF_RAMP_EPOCHS - 1)
        return float(TF_START + (TF_END - TF_START) * p)
    return float(TF_END)


def run_epoch(
    model: nn.Module,
    loader: DataLoader,
    nll_loss_fn: nn.Module,
    dir_loss_fn: nn.Module,
    optimizer: torch.optim.Optimizer | None = None,
    tf_ratio: float = 0.0,
) -> dict:
    is_train = optimizer is not None
    model.train(is_train)

    total = 0.0
    nll_tot = 0.0
    dir_tot = 0.0
    var_tot = 0.0
    n_items = 0

    for xb, yb_s, yb_r in loader:
        xb = xb.to(DEVICE)
        yb_s = yb_s.to(DEVICE)
        yb_r = yb_r.to(DEVICE)

        if is_train:
            optimizer.zero_grad(set_to_none=True)

        with torch.set_grad_enabled(is_train):
            mu, log_sigma, dir_logits = model(
                xb,
                y_teacher=yb_s if is_train else None,
                teacher_forcing_ratio=tf_ratio if is_train else 0.0,
            )

            nll = nll_loss_fn(mu, log_sigma, yb_s)

            dir_target = (yb_r[:, :, 3] > 0.0).float()
            dir_loss = dir_loss_fn(dir_logits, dir_target)

            # Item 5: variance matching penalty to fight collapse.
            pred_std = torch.std(mu[:, :, 3], unbiased=False)
            true_std = torch.std(yb_s[:, :, 3], unbiased=False)
            var_pen = (pred_std - true_std) ** 2

            loss = nll + DIR_LOSS_WEIGHT * dir_loss + VAR_MATCH_WEIGHT * var_pen

        if is_train:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

        bs = xb.size(0)
        total += loss.item() * bs
        nll_tot += nll.item() * bs
        dir_tot += dir_loss.item() * bs
        var_tot += var_pen.item() * bs
        n_items += bs

    return {
        'total': total / max(n_items, 1),
        'nll': nll_tot / max(n_items, 1),
        'dir': dir_tot / max(n_items, 1),
        'var': var_tot / max(n_items, 1),
    }


def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    max_epochs: int,
    patience: int,
) -> pd.DataFrame:
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, min_lr=1e-5)

    nll_loss_fn = WeightedGaussianNLL(NLL_WEIGHTS).to(DEVICE)
    dir_loss_fn = nn.BCEWithLogitsLoss().to(DEVICE)

    best_val = float('inf')
    best_state = copy.deepcopy(model.state_dict())
    wait = 0
    rows = []

    for epoch in range(1, max_epochs + 1):
        tf = tf_ratio_for_epoch(epoch)

        tr = run_epoch(model, train_loader, nll_loss_fn, dir_loss_fn, optimizer=optimizer, tf_ratio=tf)
        va = run_epoch(model, val_loader, nll_loss_fn, dir_loss_fn, optimizer=None, tf_ratio=0.0)

        scheduler.step(va['total'])
        lr = optimizer.param_groups[0]['lr']

        rows.append({
            'epoch': epoch,
            'tf_ratio': tf,
            'lr': lr,
            'train_total': tr['total'],
            'val_total': va['total'],
            'train_nll': tr['nll'],
            'val_nll': va['nll'],
            'train_dir': tr['dir'],
            'val_dir': va['dir'],
            'train_var': tr['var'],
            'val_var': va['var'],
        })

        print(
            f"Epoch {epoch:02d} | tf={tf:.3f} | "
            f"train={tr['total']:.5f} (nll={tr['nll']:.5f}, dir={tr['dir']:.5f}, var={tr['var']:.5f}) | "
            f"val={va['total']:.5f} (nll={va['nll']:.5f}, dir={va['dir']:.5f}, var={va['var']:.5f}) | lr={lr:.6g}"
        )

        if va['total'] < best_val:
            best_val = va['total']
            best_state = copy.deepcopy(model.state_dict())
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f'Early stopping at epoch {epoch}.')
                break

    model.load_state_dict(best_state)
    return pd.DataFrame(rows)


In [ ]:
# Metrics and fold execution
def rmse(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.sqrt(np.mean((np.asarray(a) - np.asarray(b)) ** 2)))


def directional_accuracy_eps(actual_close: np.ndarray, pred_close: np.ndarray, prev_close: np.ndarray, eps: float) -> float:
    am = np.asarray(actual_close) - np.asarray(prev_close)
    pm = np.asarray(pred_close) - np.asarray(prev_close)
    mask = np.abs(am) > eps
    if not np.any(mask):
        return float('nan')
    return float(np.mean(np.sign(am[mask]) == np.sign(pm[mask])))


def evaluate_metrics(actual_ohlc: np.ndarray, pred_ohlc: np.ndarray, prev_close: np.ndarray) -> dict:
    actual_ohlc = np.asarray(actual_ohlc, dtype=np.float32)
    pred_ohlc = np.asarray(pred_ohlc, dtype=np.float32)
    prev_close = np.asarray(prev_close, dtype=np.float32)

    ac = actual_ohlc[:, 3]
    pc = pred_ohlc[:, 3]

    return {
        'close_mae': float(np.mean(np.abs(ac - pc))),
        'close_rmse': rmse(ac, pc),
        'ohlc_mae': float(np.mean(np.abs(actual_ohlc - pred_ohlc))),
        'ohlc_rmse': rmse(actual_ohlc.reshape(-1), pred_ohlc.reshape(-1)),
        'directional_accuracy_eps': directional_accuracy_eps(ac, pc, prev_close, DIRECTION_EPS),
        'mean_signed_bias': float(np.mean(pc - ac)),
    }


def evaluate_baselines(actual_ohlc: np.ndarray, prev_ohlc: np.ndarray, prev_close: np.ndarray) -> dict:
    persistence = evaluate_metrics(actual_ohlc, prev_ohlc, prev_close)
    flat = np.repeat(prev_close.reshape(-1, 1), 4, axis=1).astype(np.float32)
    flat_rw = evaluate_metrics(actual_ohlc, flat, prev_close)
    return {'persistence': persistence, 'flat_close_rw': flat_rw}


@torch.no_grad()
def predict_multistep_mean(model: nn.Module, X: np.ndarray, batch_size: int = 512) -> tuple[np.ndarray, np.ndarray]:
    model.eval()
    mus = []
    logs = []
    for i in range(0, len(X), batch_size):
        xb = torch.from_numpy(X[i : i + batch_size]).float().to(DEVICE)
        mu, log_sigma, _ = model(xb, y_teacher=None, teacher_forcing_ratio=0.0)
        mus.append(mu.cpu().numpy())
        logs.append(log_sigma.cpu().numpy())
    return np.concatenate(mus, axis=0).astype(np.float32), np.concatenate(logs, axis=0).astype(np.float32)


@torch.no_grad()
def recursive_forecast_features(
    model: nn.Module,
    seed_window: np.ndarray,
    horizon: int,
    scale_mean: np.ndarray,
    scale_std: np.ndarray,
    clip_low: np.ndarray,
    clip_high: np.ndarray,
    sample: bool,
    temperature: float,
) -> np.ndarray:
    model.eval()
    w = seed_window.copy().astype(np.float32)
    out = []

    for _ in range(horizon):
        xb = torch.from_numpy(w).float().unsqueeze(0).to(DEVICE)
        mu_s, log_s, _ = model(xb, y_teacher=None, teacher_forcing_ratio=0.0)

        mu_s = mu_s[:, 0, :].cpu().numpy()[0]
        log_s = log_s[:, 0, :].cpu().numpy()[0]

        mu_raw = undo_standardizer(mu_s.reshape(1, -1), scale_mean, scale_std)[0]
        sigma_raw = (np.exp(log_s) * scale_std).astype(np.float32)

        if sample:
            step_raw = mu_raw + np.random.normal(loc=0.0, scale=1.0, size=mu_raw.shape).astype(np.float32) * sigma_raw * float(temperature)
        else:
            step_raw = mu_raw

        # Item 5: relaxed clipping, mainly OHLC.
        if APPLY_CLIPPING:
            step_raw[:4] = np.clip(step_raw[:4], clip_low[:4], clip_high[:4])
            if CLIP_NON_OHLC:
                step_raw[4:] = np.clip(step_raw[4:], clip_low[4:], clip_high[4:])

        step_scaled = apply_standardizer(step_raw.reshape(1, -1), scale_mean, scale_std)[0]

        # Input-only feature: rolling imputed fraction proxy (keep continuity).
        next_imp_frac = float(np.mean(w[:, -1]))
        next_in = np.concatenate([step_scaled, np.array([next_imp_frac], dtype=np.float32)]).astype(np.float32)

        out.append(step_raw.astype(np.float32))
        w = np.vstack([w[1:], next_in]).astype(np.float32)

    return np.asarray(out, dtype=np.float32)


def evaluate_recursive_by_horizon(
    model: nn.Module,
    base_scaled: np.ndarray,
    row_imputed: np.ndarray,
    price_values: np.ndarray,
    prev_close_values: np.ndarray,
    anchors: list[int],
    window: int,
    horizon: int,
    scale_mean: np.ndarray,
    scale_std: np.ndarray,
    clip_low: np.ndarray,
    clip_high: np.ndarray,
) -> pd.DataFrame:
    m_err = [[] for _ in range(horizon)]
    p_err = [[] for _ in range(horizon)]
    f_err = [[] for _ in range(horizon)]

    for i in anchors:
        imp_frac = float(row_imputed[i - window : i].mean())
        seed = np.concatenate([base_scaled[i - window : i], np.full((window, 1), imp_frac, dtype=np.float32)], axis=1)

        last_close = float(prev_close_values[i])

        pred_feat = recursive_forecast_features(
            model=model,
            seed_window=seed,
            horizon=horizon,
            scale_mean=scale_mean,
            scale_std=scale_std,
            clip_low=clip_low,
            clip_high=clip_high,
            sample=False,
            temperature=1.0,
        )

        pred_price = returns_to_prices_seq(pred_feat[:, :4], last_close=last_close)
        actual = price_values[i + 1 : i + 1 + horizon]

        persist_seq = np.repeat(price_values[i : i + 1], horizon, axis=0)
        c0 = prev_close_values[i]
        flat_seq = np.repeat(np.array([[c0, c0, c0, c0]], dtype=np.float32), horizon, axis=0)

        for h in range(horizon):
            m_err[h].append(abs(float(pred_price[h, 3] - actual[h, 3])))
            p_err[h].append(abs(float(persist_seq[h, 3] - actual[h, 3])))
            f_err[h].append(abs(float(flat_seq[h, 3] - actual[h, 3])))

    rows = []
    for h in range(horizon):
        rows.append({
            'horizon': h + 1,
            'model_close_mae': float(np.mean(m_err[h])) if m_err[h] else np.nan,
            'persistence_close_mae': float(np.mean(p_err[h])) if p_err[h] else np.nan,
            'flat_close_mae': float(np.mean(f_err[h])) if f_err[h] else np.nan,
        })
    return pd.DataFrame(rows)


def overfit_sanity_check(X_train: np.ndarray, y_train_s: np.ndarray, y_train_r: np.ndarray, output_dim: int) -> dict:
    n = min(1000, len(X_train))
    if n < 300:
        return {'ran': False, 'reason': 'too_few_samples'}

    ds = MultiStepDataset(X_train[:n], y_train_s[:n], y_train_r[:n])
    dl = DataLoader(ds, batch_size=128, shuffle=True, drop_last=False)

    model = Seq2SeqProbGRU(
        input_size=X_train.shape[-1],
        output_size=output_dim,
        hidden_size=128,
        num_layers=1,
        dropout=0.0,
        horizon=HORIZON,
    ).to(DEVICE)

    nll_loss_fn = WeightedGaussianNLL(NLL_WEIGHTS).to(DEVICE)
    dir_loss_fn = nn.BCEWithLogitsLoss().to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=0.0)

    init_stats = run_epoch(model, dl, nll_loss_fn, dir_loss_fn, optimizer=None, tf_ratio=0.0)
    for _ in range(15):
        _ = run_epoch(model, dl, nll_loss_fn, dir_loss_fn, optimizer=opt, tf_ratio=1.0)
    fin_stats = run_epoch(model, dl, nll_loss_fn, dir_loss_fn, optimizer=None, tf_ratio=0.0)

    passed = bool(fin_stats['total'] < init_stats['total'] * 0.65)
    return {'ran': True, 'initial_total': float(init_stats['total']), 'final_total': float(fin_stats['total']), 'passed': passed}


def run_fold(
    fold_name: str,
    price_fold: pd.DataFrame,
    window: int,
    max_epochs: int,
    patience: int,
    run_sanity: bool = False,
    quick_mode: bool = False,
) -> dict:
    feat_fold = build_feature_frame(price_fold)

    base_raw = feat_fold[BASE_FEATURE_COLS].to_numpy(np.float32)
    row_imputed = feat_fold['row_imputed'].to_numpy(np.int8).astype(bool)
    prev_close = feat_fold['prev_close'].to_numpy(np.float32)

    price_vals = price_fold[OHLC_COLS].to_numpy(np.float32)

    tr_end, va_end = split_points(len(base_raw))
    if tr_end <= window or va_end <= tr_end:
        raise RuntimeError(f'{fold_name}: invalid split points for rows={len(base_raw)}, window={window}')

    mean, std = fit_standardizer(base_raw[:tr_end])
    base_scaled = apply_standardizer(base_raw, mean, std)

    ql, qh = CLIP_QUANTILES
    clip_low = np.quantile(base_raw[:tr_end], ql, axis=0).astype(np.float32)
    clip_high = np.quantile(base_raw[:tr_end], qh, axis=0).astype(np.float32)

    X_all, y_all_s, y_all_r, starts, prev_close_starts, dropped_target_imputed = make_multistep_windows(
        base_scaled=base_scaled,
        base_raw=base_raw,
        row_imputed=row_imputed,
        starts_prev_close=prev_close,
        window=window,
        horizon=HORIZON,
    )

    if len(X_all) == 0:
        raise RuntimeError(f'{fold_name}: no windows available after target-imputation filtering.')

    end_idx = starts + HORIZON - 1
    tr_m = end_idx < tr_end
    va_m = (end_idx >= tr_end) & (end_idx < va_end)
    te_m = end_idx >= va_end

    X_train, y_train_s, y_train_r = X_all[tr_m], y_all_s[tr_m], y_all_r[tr_m]
    X_val, y_val_s, y_val_r = X_all[va_m], y_all_s[va_m], y_all_r[va_m]
    X_test, y_test_s, y_test_r = X_all[te_m], y_all_s[te_m], y_all_r[te_m]
    test_starts = starts[te_m]
    test_prev_close = prev_close_starts[te_m]

    if min(len(X_train), len(X_val), len(X_test)) == 0:
        raise RuntimeError(f'{fold_name}: empty train/val/test split after windowing.')

    sanity = overfit_sanity_check(X_train, y_train_s, y_train_r, output_dim=len(BASE_FEATURE_COLS)) if run_sanity else {'ran': False}

    train_loader = DataLoader(MultiStepDataset(X_train, y_train_s, y_train_r), batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
    val_loader = DataLoader(MultiStepDataset(X_val, y_val_s, y_val_r), batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

    model = Seq2SeqProbGRU(
        input_size=X_train.shape[-1],
        output_size=len(BASE_FEATURE_COLS),
        hidden_size=HIDDEN_SIZE,
        num_layers=NUM_LAYERS,
        dropout=DROPOUT,
        horizon=HORIZON,
    ).to(DEVICE)

    hist = train_model(model, train_loader, val_loader, max_epochs=max_epochs, patience=patience)

    mu_test_s, log_test_s = predict_multistep_mean(model, X_test, batch_size=512)
    mu_test_raw = undo_standardizer(mu_test_s.reshape(-1, len(BASE_FEATURE_COLS)), mean, std).reshape(mu_test_s.shape)

    pred_step1_ret = mu_test_raw[:, 0, :4]
    actual_step1_ret = y_test_r[:, 0, :4]

    actual_ohlc_1 = price_vals[test_starts + 1]
    pred_ohlc_1 = one_step_returns_to_prices_batch(pred_step1_ret, test_prev_close)

    prev_ohlc = price_vals[test_starts]

    model_metrics = evaluate_metrics(actual_ohlc_1, pred_ohlc_1, test_prev_close)
    baseline_metrics = evaluate_baselines(actual_ohlc_1, prev_ohlc, test_prev_close)

    pred_close_std = float(np.std(pred_step1_ret[:, 3]))
    actual_close_std = float(np.std(actual_step1_ret[:, 3]))
    std_ratio = float(pred_close_std / max(actual_close_std, 1e-12))

    if quick_mode:
        return {
            'fold': fold_name,
            'window': int(window),
            'history_df': hist,
            'sanity': sanity,
            'model_metrics': model_metrics,
            'baseline_metrics': baseline_metrics,
            'pred_actual_std_ratio': std_ratio,
            'samples': {
                'train': int(len(X_train)),
                'val': int(len(X_val)),
                'test': int(len(X_test)),
                'dropped_target_imputed': int(dropped_target_imputed),
            },
        }

    anchors = [int(i) for i in test_starts if i >= window and (i + HORIZON) < len(price_vals)]
    if not anchors:
        raise RuntimeError(f'{fold_name}: no valid anchors for recursive evaluation.')

    horizon_df = evaluate_recursive_by_horizon(
        model=model,
        base_scaled=base_scaled,
        row_imputed=row_imputed,
        price_values=price_vals,
        prev_close_values=prev_close,
        anchors=anchors,
        window=window,
        horizon=HORIZON,
        scale_mean=mean,
        scale_std=std,
        clip_low=clip_low,
        clip_high=clip_high,
    )

    last_anchor = anchors[-1]
    imp_frac = float(row_imputed[last_anchor - window : last_anchor].mean())
    seed = np.concatenate([
        base_scaled[last_anchor - window : last_anchor],
        np.full((window, 1), imp_frac, dtype=np.float32),
    ], axis=1)

    last_close = float(prev_close[last_anchor])

    pred_feat_det = recursive_forecast_features(
        model=model,
        seed_window=seed,
        horizon=HORIZON,
        scale_mean=mean,
        scale_std=std,
        clip_low=clip_low,
        clip_high=clip_high,
        sample=False,
        temperature=1.0,
    )
    pred_price_det = returns_to_prices_seq(pred_feat_det[:, :4], last_close=last_close)

    pred_feat_sample = recursive_forecast_features(
        model=model,
        seed_window=seed,
        horizon=HORIZON,
        scale_mean=mean,
        scale_std=std,
        clip_low=clip_low,
        clip_high=clip_high,
        sample=True,
        temperature=INFER_SAMPLE_TEMPERATURE,
    )
    pred_price_sample = returns_to_prices_seq(pred_feat_sample[:, :4], last_close=last_close)

    actual_price_path = price_vals[last_anchor + 1 : last_anchor + 1 + HORIZON]
    future_idx = price_fold.index[last_anchor + 1 : last_anchor + 1 + HORIZON]

    pred_future_df_det = pd.DataFrame(pred_price_det, index=future_idx, columns=OHLC_COLS)
    pred_future_df_sample = pd.DataFrame(pred_price_sample, index=future_idx, columns=OHLC_COLS)
    actual_future_df = pd.DataFrame(actual_price_path, index=future_idx, columns=OHLC_COLS)

    known_pos = last_anchor
    context_df = price_fold.iloc[max(0, known_pos - 169) : known_pos + 1][OHLC_COLS].copy()

    step15 = horizon_df[horizon_df['horizon'] == HORIZON].iloc[0]

    return {
        'fold': fold_name,
        'window': int(window),
        'history_df': hist,
        'sanity': sanity,
        'model_metrics': model_metrics,
        'baseline_metrics': baseline_metrics,
        'horizon_df': horizon_df,
        'pred_actual_std_ratio': std_ratio,
        'samples': {
            'train': int(len(X_train)),
            'val': int(len(X_val)),
            'test': int(len(X_test)),
            'dropped_target_imputed': int(dropped_target_imputed),
            'anchors': int(len(anchors)),
        },
        'step15': {
            'model_close_mae': float(step15['model_close_mae']),
            'persistence_close_mae': float(step15['persistence_close_mae']),
            'flat_close_mae': float(step15['flat_close_mae']),
        },
        'context_df': context_df,
        'actual_future_df': actual_future_df,
        'pred_future_df_det': pred_future_df_det,
        'pred_future_df_sample': pred_future_df_sample,
    }


In [ ]:
# Run lookback sweep then full walk-forward
fold_results = []

primary_slice = slices[0]
sweep_rows = []
selected_window = DEFAULT_LOOKBACK

if ENABLE_LOOKBACK_SWEEP:
    print('\n=== Lookback sweep (quick mode on first fold) ===')
    _, a0, b0 = primary_slice
    fold_price0 = price_df.iloc[a0:b0].copy()

    for w in LOOKBACK_CANDIDATES:
        print(f'\n-- Sweep candidate lookback={w} --')
        r = run_fold(
            fold_name=f'sweep_w{w}',
            price_fold=fold_price0,
            window=w,
            max_epochs=SWEEP_MAX_EPOCHS,
            patience=SWEEP_PATIENCE,
            run_sanity=False,
            quick_mode=True,
        )

        m = r['model_metrics']
        p = r['baseline_metrics']['persistence']
        std_ratio = r['pred_actual_std_ratio']

        # Higher is better: beat persistence and increase variance ratio.
        score = (p['close_mae'] - m['close_mae']) + 0.15 * std_ratio

        sweep_rows.append({
            'lookback': w,
            'model_close_mae': m['close_mae'],
            'persistence_close_mae': p['close_mae'],
            'std_ratio': std_ratio,
            'score': score,
            'dropped_target_imputed': r['samples']['dropped_target_imputed'],
        })

    sweep_df = pd.DataFrame(sweep_rows).sort_values('score', ascending=False).reset_index(drop=True)
    display(sweep_df)

    selected_window = int(sweep_df.iloc[0]['lookback'])
    print(f'Selected lookback from sweep: {selected_window}')
else:
    sweep_df = pd.DataFrame()
    selected_window = DEFAULT_LOOKBACK

print('\n=== Full walk-forward with selected lookback ===')
for i, (name, a, b) in enumerate(slices, start=1):
    print(f'\n=== Running {name} [{a}:{b}] lookback={selected_window} ===')
    fold_price = price_df.iloc[a:b].copy()

    res = run_fold(
        fold_name=name,
        price_fold=fold_price,
        window=selected_window,
        max_epochs=FINAL_MAX_EPOCHS,
        patience=FINAL_PATIENCE,
        run_sanity=(i == 1),
        quick_mode=False,
    )
    fold_results.append(res)

    print('Samples:', res['samples'])
    print('Sanity:', res['sanity'])
    print('One-step model:', res['model_metrics'])
    print('One-step persistence:', res['baseline_metrics']['persistence'])
    print('One-step flat:', res['baseline_metrics']['flat_close_rw'])
    print('Recursive step-15:', res['step15'])
    print('Std ratio (pred/actual close return):', res['pred_actual_std_ratio'])

if not fold_results:
    raise RuntimeError('No fold results produced.')

summary_rows = []
for r in fold_results:
    m = r['model_metrics']
    p = r['baseline_metrics']['persistence']
    f = r['baseline_metrics']['flat_close_rw']
    s15 = r['step15']

    summary_rows.append({
        'fold': r['fold'],
        'window': r['window'],
        'model_close_mae': m['close_mae'],
        'model_directional_acc_eps': m['directional_accuracy_eps'],
        'model_bias': m['mean_signed_bias'],
        'persist_close_mae': p['close_mae'],
        'flat_close_mae': f['close_mae'],
        'step15_model_mae': s15['model_close_mae'],
        'step15_persist_mae': s15['persistence_close_mae'],
        'pred_actual_std_ratio': r['pred_actual_std_ratio'],
        'dropped_target_imputed': r['samples']['dropped_target_imputed'],
    })

summary_df = pd.DataFrame(summary_rows)
display(summary_df)

metric_cols = [c for c in summary_df.columns if c not in {'fold'}]
agg_df = pd.DataFrame({'mean': summary_df[metric_cols].mean(), 'std': summary_df[metric_cols].std(ddof=0)})
display(agg_df)

horizon_all = pd.concat([r['horizon_df'].assign(fold=r['fold']) for r in fold_results], ignore_index=True)
horizon_stats = horizon_all.groupby('horizon', as_index=False)[['model_close_mae', 'persistence_close_mae', 'flat_close_mae']].mean()
display(horizon_stats)


In [ ]:
# Acceptance checks + diagnostics
latest = fold_results[-1]

mean_model_mae = float(summary_df['model_close_mae'].mean())
mean_persist_mae = float(summary_df['persist_close_mae'].mean())
mean_dir = float(summary_df['model_directional_acc_eps'].mean())
mean_bias = float(summary_df['model_bias'].mean())
mean_step15_model = float(summary_df['step15_model_mae'].mean())
mean_step15_persist = float(summary_df['step15_persist_mae'].mean())
mean_std_ratio = float(summary_df['pred_actual_std_ratio'].mean())

pred_close_path = latest['pred_future_df_sample']['Close'].to_numpy()
is_monotonic = bool(np.all(np.diff(pred_close_path) >= 0) or np.all(np.diff(pred_close_path) <= 0))

acceptance = {
    'criterion_1_model_mae_20pct_better_than_persistence': mean_model_mae <= 0.8 * mean_persist_mae,
    'criterion_2_directional_accuracy_eps_at_least_0_52': mean_dir >= 0.52,
    'criterion_3_step15_better_than_persistence': mean_step15_model < mean_step15_persist,
    'criterion_4_abs_bias_within_25pct_of_mae': abs(mean_bias) <= 0.25 * mean_model_mae,
    'criterion_5_non_monotonic_latest_prediction_path': not is_monotonic,
    'criterion_6_prediction_variance_not_collapsed': mean_std_ratio >= STD_RATIO_TARGET_MIN,
}

print('Selected lookback:', int(summary_df['window'].iloc[0]))
print('Acceptance checks:')
for k, v in acceptance.items():
    print(f'  {k}: {v}')

fig, axes = plt.subplots(1, 3, figsize=(18, 4.8), facecolor='white')

hist = latest['history_df']
axes[0].plot(hist['epoch'], hist['train_total'], label='Train total', color='black')
axes[0].plot(hist['epoch'], hist['val_total'], label='Val total', color='gray')
axes[0].plot(hist['epoch'], hist['val_nll'], label='Val NLL', color='#1f77b4', alpha=0.8)
axes[0].plot(hist['epoch'], hist['val_dir'], label='Val Dir', color='#d62728', alpha=0.8)
axes[0].set_title(f"Loss Curves ({latest['fold']})")
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].grid(alpha=0.25)
axes[0].legend()

axes[1].plot(horizon_stats['horizon'], horizon_stats['model_close_mae'], label='Model', color='black', linewidth=2.0)
axes[1].plot(horizon_stats['horizon'], horizon_stats['persistence_close_mae'], label='Persistence', color='#E74C3C')
axes[1].plot(horizon_stats['horizon'], horizon_stats['flat_close_mae'], label='Flat RW', color='#3498DB')
axes[1].set_title('Recursive Close MAE by Horizon')
axes[1].set_xlabel('Horizon')
axes[1].set_ylabel('MAE')
axes[1].grid(alpha=0.25)
axes[1].legend()

axes[2].bar(summary_df['fold'], summary_df['pred_actual_std_ratio'], color='#555555')
axes[2].axhline(1.0, color='#2ca02c', linestyle='--', linewidth=1.2, label='ideal=1.0')
axes[2].axhline(STD_RATIO_TARGET_MIN, color='#ff7f0e', linestyle=':', linewidth=1.2, label=f'min target={STD_RATIO_TARGET_MIN}')
axes[2].set_title('Predicted/Actual Close Return Std Ratio')
axes[2].set_ylabel('Std ratio')
axes[2].grid(alpha=0.25)
axes[2].legend()

plt.tight_layout()
plt.show()


In [ ]:
# Final candlestick chart (history green/red, predicted white/black)
def draw_candles(
    ax,
    ohlc: pd.DataFrame,
    start_x: int,
    up_edge: str,
    up_face: str,
    down_edge: str,
    down_face: str,
    wick_color: str,
    width: float = 0.62,
    lw: float = 1.0,
    alpha: float = 1.0,
):
    vals = ohlc[OHLC_COLS].to_numpy()
    for i, (o, h, l, c) in enumerate(vals):
        x = start_x + i
        bull = c >= o

        ax.vlines(x, l, h, color=wick_color, linewidth=lw, alpha=alpha, zorder=2)

        lower = min(o, c)
        height = abs(c - o)
        if height < 1e-8:
            height = 1e-6

        rect = Rectangle(
            (x - width / 2, lower),
            width,
            height,
            facecolor=up_face if bull else down_face,
            edgecolor=up_edge if bull else down_edge,
            linewidth=lw,
            alpha=alpha,
            zorder=3,
        )
        ax.add_patch(rect)


context_df = latest['context_df']
actual_future_df = latest['actual_future_df']
pred_future_df = latest['pred_future_df_sample']

fig, ax = plt.subplots(figsize=(17, 8), facecolor='black')
ax.set_facecolor('black')

draw_candles(ax, context_df, 0, '#2ECC71', '#2ECC71', '#E74C3C', '#E74C3C', '#DADADA', width=0.58, lw=1.0, alpha=0.95)
draw_candles(ax, actual_future_df, len(context_df), '#1D6F42', '#1D6F42', '#8E2F25', '#8E2F25', '#9A9A9A', width=0.58, lw=1.0, alpha=0.70)
draw_candles(ax, pred_future_df, len(context_df), '#FFFFFF', '#FFFFFF', '#000000', '#000000', '#F5F5F5', width=0.50, lw=1.35, alpha=1.0)

ax.axvline(len(context_df) - 0.5, color='white', linestyle='--', linewidth=0.9, alpha=0.6)

idx = context_df.index.append(actual_future_df.index)
n = len(idx)
step = max(1, n // 10)
ticks = list(range(0, n, step))
if ticks[-1] != n - 1:
    ticks.append(n - 1)

labels = [idx[i].strftime('%m-%d %H:%M') for i in ticks]
ax.set_xticks(ticks)
ax.set_xticklabels(labels, rotation=26, ha='right', color='white', fontsize=9)

ax.tick_params(axis='y', colors='white')
for sp in ax.spines.values():
    sp.set_color('#666666')

ax.grid(color='#252525', linewidth=0.6, alpha=0.35)
ax.set_title(f'MSFT 1m ({latest["fold"]}) - History vs 15-step Forecast', color='white', pad=14)
ax.set_ylabel('Price', color='white')

legend_elements = [
    Patch(facecolor='#2ECC71', edgecolor='#2ECC71', label='History bullish (green)'),
    Patch(facecolor='#E74C3C', edgecolor='#E74C3C', label='History bearish (red)'),
    Patch(facecolor='#FFFFFF', edgecolor='#FFFFFF', label='Predicted bullish (white)'),
    Patch(facecolor='#000000', edgecolor='#FFFFFF', label='Predicted bearish (black)'),
]
leg = ax.legend(handles=legend_elements, facecolor='black', edgecolor='#707070', framealpha=1.0, loc='upper left')
for t in leg.get_texts():
    t.set_color('white')

plt.tight_layout()
plt.show()


## Notes

- If `pred_actual_std_ratio` remains low, the model is still volatility-collapsed.
- The chart now displays the sampled recursive path (`temperature` controlled), not only deterministic mean path.
- For stricter realism, next step is quote-level order-book imbalance features from NBBO/trades data.
